In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_val_predict, cross_val_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import joblib

In [2]:
# 0 - Charger les données
data = pd.read_csv('../data/processed/merged_data_2019_2022.csv')


In [3]:
# 1 - Découper le jeu de données en k-folds et sauvegarder les indices des plis pour la reproductibilité
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
fold_indices = list(kfold.split(X=data.drop(columns=['grav']), y=data['grav']))
joblib.dump(fold_indices, '../data/fold_indices.pkl')

['../data/fold_indices.pkl']

In [4]:
# Charger les indices des plis enregistrés
fold_indices = joblib.load('../data/fold_indices.pkl')

In [5]:
# 2 - Équilibrer la variable 'grav' entre chaque modalité et préparer le pipeline
X = data.drop(columns=['grav'])
y = data['grav']

In [6]:
# Sélection des colonnes numériques et catégorielles
numeric_features = make_column_selector(dtype_include=['int64', 'float64'])
categorical_features = make_column_selector(dtype_include=['object'])

In [7]:
# Prétraitement : MinMaxScaler pour les colonnes numériques, OneHotEncoder pour les colonnes catégorielles
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ]
)

In [8]:
# Liste des classificateurs à tester
classifiers = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Naive Bayes': GaussianNB(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

In [9]:
# Paramètres des hyperparamètres pour chaque modèle
param_grids = {
    'Logistic Regression': {
        'classifier__C': [0.1, 1, 10]
    },
    'SVM': {
        'classifier__C': [0.1, 1, 10],
        'classifier__kernel': ['linear', 'rbf']
    },
    'Decision Tree': {
        'classifier__max_depth': [5, 10, 20],
        'classifier__min_samples_split': [2, 5, 10]
    },
    'Random Forest': {
        'classifier__n_estimators': [50, 100],
        'classifier__max_depth': [10, 20],
        'classifier__min_samples_split': [2, 5],
        'classifier__min_samples_leaf': [1, 2]
    },
    'Naive Bayes': {},
    'XGBoost': {
        'classifier__n_estimators': [50, 100],
        'classifier__learning_rate': [0.01, 0.1, 0.2],
        'classifier__max_depth': [3, 5, 10]
    }
}

In [ ]:
# Boucle pour tester chaque classificateur
for name, classifier in classifiers.items():
    print(f"\nTesting model: {name}")
    
    # Créer un pipeline pour équilibrer les classes, encoder les colonnes, appliquer PCA et entraîner le modèle
    pipeline = ImbPipeline([
        ('sampling', RandomUnderSampler(sampling_strategy='auto')),
        ('preprocessing', preprocessor),
        ('pca', PCA(n_components=0.95)),
        ('classifier', classifier)
    ])
    
    # Obtenir les paramètres d'hyperparamètres pour le classificateur actuel
    param_distributions = param_grids.get(name, {})
    
    # Effectuer une recherche des meilleurs hyperparamètres avec RandomizedSearchCV
    if param_distributions:
        search = RandomizedSearchCV(pipeline, param_distributions, n_iter=10, cv=kfold, scoring='accuracy', n_jobs=1, random_state=42)
    else:
        # Pas d'hyperparamètres à optimiser, utiliser simplement la validation croisée
        search = pipeline
    
    # Effectuer la validation croisée avec prédictions
    y_pred = cross_val_predict(search, X, y, cv=kfold)
    
    # Générer la matrice de confusion
    cm = confusion_matrix(y, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues')
    plt.title(f"Matrice de confusion pour {name}")
    plt.show()
    
    # Afficher les résultats de la validation croisée
    if param_distributions:
        search.fit(X, y)
        print("Meilleurs paramètres :", search.best_params_)
        print("Meilleur score :", search.best_score_)
    else:
        print("Score moyen (validation croisée) :", np.mean(cross_val_score(search, X, y, cv=kfold, scoring='accuracy')))



Testing model: Logistic Regression


E:\Telechargements\Datascientest\Git_GitHub\repos\Projet_Accidentologie\venv\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
E:\Telechargements\Datascientest\Git_GitHub\repos\Projet_Accidentologie\venv\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
